In [24]:
#libraries
import torch
import torch.nn as nn
import random
from IPython import get_ipython
from IPython.display import display
!pip install transformers
from transformers import AutoTokenizer
import torch.optim as optim

In [29]:
#transformer model with updating it with mask part
class TransformerLM(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers):
        super(TransformerLM, self).__init__()
        self.mask_token = "[MASK]"
        self.vocab_size = vocab_size + 1  # Add mask token to vocabulary
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc = nn.Linear(d_model, self.vocab_size)

    def mask_input(self, input_ids):
        # Convert input_ids back to tokens for printing
        tokens = tokenizer.convert_ids_to_tokens(input_ids)
        print("Original Tokens:", tokens)  # Print the original tokens

        num_tokens = len(input_ids)
        num_to_mask = int(0.2 * num_tokens)  # Calculate 20% of tokens

        masked_indices = random.sample(range(num_tokens), num_to_mask)  # Randomly select indices
        labels = [input_ids[i] for i in masked_indices]  # Get true labels for masked tokens

        for index in masked_indices:
            input_ids[index] = self.vocab_size - 1  # Replace with mask token ID

        # Convert masked input_ids back to tokens for printing
        masked_tokens = tokenizer.convert_ids_to_tokens(input_ids)
        print("Masked Tokens:", masked_tokens)  # Print the masked tokens for visualizes

        return input_ids, masked_indices, labels

    def forward(self, input_ids, masked_indices):
        embeds = self.embedding(input_ids)

        output = self.transformer(embeds, embeds)
        output = self.fc(output)
        # Select masked outputs correctly
        masked_outputs = output[0, masked_indices, :]
        return masked_outputs

    def compute_loss(self, masked_outputs, labels):
        # Convert labels to tensor and ensure Long type
        labels = torch.tensor(labels, dtype=torch.long)

        criterion = nn.CrossEntropyLoss()
        loss = criterion(masked_outputs, labels)
        return loss


In [31]:
#example
text = "The quick brown fox jumps over the lazy dog."


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize(text)
input_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids_masked, masked_indices, labels = model.mask_input(input_ids.copy())
input_ids_masked = torch.tensor([input_ids_masked])

# Model and optimizer
model = TransformerLM(tokenizer.vocab_size, d_model=512, nhead=8, num_layers=6)
optimizer = optim.Adam(model.parameters(), lr=0.001)


optimizer.zero_grad()
masked_outputs = model(input_ids_masked, masked_indices)
loss = model.compute_loss(masked_outputs, labels)

# Get predictions and scores (only if there were masked tokens)
if loss is not None:
    _, predicted_indices = torch.max(masked_outputs, dim=1)
    scores = [masked_outputs[i, labels[i]].item() for i in range(len(predicted_indices))]

    # Print masked token indices and scores
    for index, score in zip(masked_indices, scores):
        print(f"Masked token index: {index}, Score: {score:.4f}")

    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    print(f"Loss: {loss.item():.4f}")

Original Tokens: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']
Masked Tokens: ['the', 'quick', 'brown', 'fox', None, 'over', 'the', 'lazy', None, '.']
Masked token index: 8, Score: -0.7705
Masked token index: 4, Score: -0.2806
Loss: 11.0232
